#PREPROCESSING

##IMPORT DATASET

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv', sep=',')

##ENCODING

In [ ]:
# Frequency encoding for origin_host
origin_host_frequency = df['origin_host'].value_counts()
df['origin_host'] = df['origin_host'].map(origin_host_frequency)

# Frequency encoding for response_host
response_host_frequency = df['response_host'].value_counts()
df['response_host'] = df['response_host'].map(response_host_frequency)

##IMPUTE NAN VALUE

In [ ]:
df_impute = df.drop(columns = ['id', 'traffic'])

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Perlu diimpor untuk mengaktifkan IterativeImputer
from sklearn.impute import IterativeImputer

# Pilih kolom numerik untuk imputasi
numeric_columns = df_impute.select_dtypes(include=['float64', 'int64']).columns

# Inisialisasi IterativeImputer
imputer = IterativeImputer(random_state=0)

# Melakukan imputasi pada kolom numerik
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

# Penanganan nilai negatif
df[numeric_columns] = df[numeric_columns].clip(lower=0)

In [ ]:
df = df.drop(columns = 'id')

##ENCODE TARGET COLUMN

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.preprocessing import StandardScaler
# Load your dataset
X = df.drop(columns='traffic')  # Replace 'traffic' with your actual feature columns
y = df['traffic']  # Replace 'traffic' with your actual target column

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_encoded = label_encoder.fit_transform(y) + 1

# Now y_encoded contains the encoded labels, and df['traffic'] still contains the original labels

##CLASS BALANCING

In [ ]:
# Apply Borderline SMOTE to balance the dataset
smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42, kind='borderline-1')  # 'kind' can be 'borderline-1' or 'borderline-2'
X_balanced, y_balanced = smote.fit_resample(X, y_encoded)

In [ ]:
# Convert to DataFrame
df_balanced = pd.DataFrame(X_balanced, columns=X.columns)
df_balanced['traffic'] = y_balanced

# Define number of classes and samples per class
num_classes = len(np.unique(y_encoded))
samples_per_class = 100000 // num_classes

# Sample equally from each class
df_subset = pd.DataFrame()

for label in np.unique(y_encoded):
    df_class = df_balanced[df_balanced['traffic'] == label]
    df_class_sampled = df_class.sample(n=samples_per_class, random_state=42)
    df_subset = pd.concat([df_subset, df_class_sampled])

# Separate features and target
X_balanced_subset = df_subset.drop(columns='traffic')
y_balanced_subset = df_subset['traffic']

##TRAIN-TEST SPLIT

In [ ]:
# Split the balanced dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced_subset, y_balanced_subset, test_size=0.2, random_state=42)

#MODELLING

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

In [ ]:
seed = 42

In [ ]:
#inisialisasi model
clf = ExtraTreesClassifier(n_estimators=100, random_state=seed)

##TANPA TRAIN-TEST SPLIT

In [ ]:
cv = KFold(n_splits=10, random_state=seed, shuffle=True)
scores = cross_val_score(clf, X_balanced_subset, y_balanced_subset, scoring='accuracy', cv=cv, n_jobs=-1)

# evaluate using cross-validation
cv = KFold(n_splits=10, random_state=seed, shuffle=True)
scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuraccy: ', mean(scores))

KeyboardInterrupt: 

##DENGAN TRAIN-TEST SPLIT (tanpa cross validation)

In [22]:
# Grid Search
from sklearn.model_selection import GridSearchCV

# Define cross-validation strategy
seed = 42
cv = KFold(n_splits=10, random_state=seed, shuffle=True)

# Define hyperparameters to tune
parameters = {
    'n_estimators' : [100, 200, 300, 500],
    'min_samples_leaf': [5,10,20],
    'max_features': [2,3,4]

}

# Initialize GridSearchCV with ExtraTreesClassifier
clf = GridSearchCV(ExtraTreesClassifier(), param_grid=parameters, cv=cv)

# Fit GridSearchCV on the training data
clf.fit(X_train, y_train)

# Print the best estimator found
print(clf.best_estimator_) #ExtraTreesClassifier(max_features=4, min_samples_leaf=5, n_estimators=500)

ExtraTreesClassifier(max_features=4, min_samples_leaf=5, n_estimators=500)


In [24]:
best_model = clf.best_estimator_
scores = cross_val_score(best_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy:', np.mean(scores))

Accuracy: 0.902045185335667


In [25]:
best_model = clf.best_estimator_
scores = cross_val_score(best_model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy:', np.mean(scores))

Accuracy: 0.8889999999999999


##DENGAN TRAIN-TEST SPLIT

#MAKE PREDICTION

In [29]:
new_data = pd.read_csv('test.csv', sep=',')
new_data.head()

,id,origin_host,origin_port,response_host,response_port,flow_duration,forward_packets_per_sec,backward_packets_per_sec,flow_packets_per_sec,down_up_ratio,...,backward_bulk_bytes,forward_bulk_packets,backward_bulk_packets,forward_bulk_rate,backward_bulk_rate,active,idle,forward_initial_window_size,backward_initial_window_size,forward_last_window_size
0,ClDOIi3yLGMfeRPzAc,103.255.15.23,49188,13.227.228.83,443,1.738424,8.628505,NaN,16.681776,NaN,...,0.0,0.0,NaN,NaN,0.000000e+00,1.738440e+06,NaN,29200.0,65535.0,80.0
1,C9chsd1cHTOBBHe6Q2,202.169.224.219,5678,255.255.255.255,5678,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,0.0,NaN,NaN,NaN,0.000000e+00,0.0,0.0,0.0,0.0
2,CUOe3U1NC5Nln7cItf,103.255.15.150,32976,8.8.8.8,53,NaN,80.446972,NaN,NaN,1.0,...,0.0,NaN,0.0,0.0,0.000000e+00,2.486110e+04,0.0,0.0,0.0,NaN
3,C6QhB04KTRUYOY2rMk,103.255.15.150,48606,128.199.242.104,443,NaN,NaN,51.891033,88.956056,NaN,...,4585.0,0.0,5.0,0.0,1.544649e+07,2.697961e+05,0.0,29200.0,NaN,NaN
4,C60oC51SZNEhCXlX6c,103.255.15.150,34794,128.199.242.104,443,0.326490,30.628816,NaN,70.446278,1.3,...,4585.0,0.0,NaN,NaN,5.211622e+07,NaN,NaN,29200.0,NaN,402.0


In [30]:
# Perform the same frequency encoding on the new data
new_data['origin_host'] = new_data['origin_host'].map(origin_host_frequency)
new_data['response_host'] = new_data['response_host'].map(response_host_frequency)

In [31]:
id = new_data['id']

In [32]:
# Drop 'id' and 'traffic' columns before imputation (if present in new data)
new_data_impute = new_data.drop(columns=['id'])

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_data)

StandardScaler()

In [38]:
# Impute missing values with the same imputer used on training data
new_data[numeric_columns] = imputer.transform(new_data_impute[numeric_columns])

# Clip negative values to 0
new_data[numeric_columns] = new_data[numeric_columns].clip(lower=0)

# # Select the same features as used in training
# new_data = new_data[selected_features]
#new_data = new_data.drop(columns=['id'])
# Standardize the features using the same scaler used for training
new_data_scaled = scaler.transform(new_data)

In [39]:
# Predict the labels using the trained model
predictions = clf.predict(new_data_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but ExtraTreesClassifier was fitted with feature names
  warnings.warn(


In [40]:
# Adjust predictions by subtracting 1
adjusted_predictions = predictions - 1

# Decode the adjusted predictions
decoded_predictions = label_encoder.inverse_transform(adjusted_predictions)
print(f"Decoded predictions: {decoded_predictions}")

Decoded predictions: ['Background' 'Benign' 'Benign' ... 'Benign' 'Benign' 'Benign']


In [41]:
# Ensure 'id' and predictions are of the same length
if len(id) == len(predictions):
    # Decode the encoded labels to original class labels
    decoded_predictions = label_encoder.inverse_transform(adjusted_predictions)

    # Create a DataFrame with 'id' and 'traffic' columns for the output
    result_df = pd.DataFrame({
        'id': id,  # Use the stored 'id' variable
        'traffic': decoded_predictions
    })

    # Save the result to a CSV file
    result_df.to_csv('predictions.csv', index=False)

    # Check the resulting DataFrame
    print(result_df.head())
else:
    print("The length of 'id' and predictions do not match.")

                   id     traffic
0  ClDOIi3yLGMfeRPzAc  Background
1  C9chsd1cHTOBBHe6Q2      Benign
2  CUOe3U1NC5Nln7cItf      Benign
3  C6QhB04KTRUYOY2rMk      Benign
4  C60oC51SZNEhCXlX6c  Background
